# ATM Cash Optimization

Agus Gunawan, Holy Lovenia

## Importing dataset

In [2]:
from datetime import datetime
from pandas import read_csv
import pandas as pd
from os import listdir, mkdir
from os.path import exists, isfile, join

### Read train data

#### Functions

In [242]:
def get_files_from_dir_path(dir_path):
    files = []
    
    for file in listdir(dir_path):
        if file.endswith('.csv'):
            files.append(file)

    return files

In [243]:
def import_all_datasets(base_src_dir_path):

    src_files = get_files_from_dir_path(base_src_dir_path)
    date_parser = lambda dates: datetime.strptime(dates, '%Y-%m-%d')
    
    datasets = {}

    for i in range(0, len(src_files)):
        current_src_file = src_files[i]
        current_id = current_src_file.split('.')[0]
        current_id = current_id.split('_')[0]
        datasets[int(current_id)] = read_csv(base_src_dir_path + current_src_file, parse_dates=['date'], date_parser=date_parser)
        datasets[int(current_id)] = datasets[int(current_id)].rename(columns={'date': 'ds', 'Withdrawals': 'y'})
        
    return datasets

In [244]:
train_datasets = import_all_datasets('dataset/train/')

len(train_datasets)

10626

In [245]:
train_datasets[1].head()

,ds,Balance.Cost,Deliveries,Returns,y,saldo awal,uang_idle
0,2018-01-01,32367,0,0,78700000,275600000,45304
1,2018-01-02,65392,460000000,146200000,112900000,196900000,32367
2,2018-01-03,47523,0,0,108700000,397800000,65392
3,2018-01-04,30970,0,0,100700000,289100000,47523
4,2018-01-05,62901,460000000,138000000,127750000,188400000,30970


## Model building

In [427]:
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation, performance_metrics
from fbprophet.plot import plot_cross_validation_metric, plot_yearly, plot_weekly

import calendar

In [428]:
gajian = pd.DataFrame({
  'holiday' : 'gajian',
  'ds' : pd.to_datetime(['2018-03-30', '2018-02-28', '2018-01-31']),
  'lower_window' : -2,
  'upper_window' : 2}
)

holidays = gajian

In [429]:
def take_money(ds):
    date = pd.to_datetime(ds)
    switcher = {
        6: 0.5
    }
    return switcher.get(date.weekday(), 1)

In [430]:
for i in range(1, len(train_datasets) + 1):
    train_datasets[i]['take_money'] = train_datasets[i]['ds'].apply(take_money)

In [431]:
prophets = {}

for i in range(1, len(train_datasets) + 1):
    prophet = Prophet(yearly_seasonality=False, 
                      weekly_seasonality=False, 
                      daily_seasonality=False, 
                      holidays=holidays)

    prophet.add_regressor(name='take_money', mode='multiplicative')
    prophet.fit(train_datasets[i])
    
    prophets[i] = prophet

/home/holy/anaconda3/envs/nlp_text/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 3.0.
/home/holy/anaconda3/envs/nlp_text/lib/python3.6/site-packages/fbprophet/forecaster.py:353: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  np.linspace(0, hist_size - 1, self.n_changepoints + 1)


In [433]:
forecast_data = {}
for i in range(1, len(prophets) + 1):
    if i % 10 == 0:
        print(i)
    future_data = prophets[i].make_future_dataframe(periods=7, freq='d')
    future_data['take_money'] = future_data['ds'].apply(take_money)
    
    forecast_data[i] = prophets[i].predict(future_data)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
222

In [435]:
pm = {}
for i in range(1, 10 + 1):
    cv = cross_validation(prophets[i], horizon='7 days')
    pm[i] = performance_metrics(cv)
    print(i, pm[i])

INFO:fbprophet.diagnostics:Making 16 forecasts with cutoffs between 2018-01-23 12:00:00 and 2018-03-17 00:00:00
INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 17.0.
/home/holy/anaconda3/envs/nlp_text/lib/python3.6/site-packages/fbprophet/forecaster.py:353: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  np.linspace(0, hist_size - 1, self.n_changepoints + 1)
/home/holy/anaconda3/envs/nlp_text/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 20.0.
INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 23.0.
INFO:fbprophet.diagnostics:Making 16 forecasts with cutof

1             horizon           mse          rmse           mae      mape  \
35  1 days 00:00:00  1.204326e+15  3.470339e+07  2.652938e+07  0.218945   
63  1 days 00:00:00  1.126552e+15  3.356415e+07  2.414921e+07  0.194172   
77  1 days 00:00:00  1.105918e+15  3.325535e+07  2.311800e+07  0.185707   
105 1 days 00:00:00  9.192885e+14  3.031977e+07  1.951526e+07  0.154925   
21  1 days 00:00:00  7.992215e+14  2.827051e+07  1.659158e+07  0.146958   
91  1 days 00:00:00  1.020820e+14  1.010356e+07  8.692134e+06  0.113619   
15  1 days 12:00:00  1.378558e+14  1.174120e+07  9.805787e+06  0.117029   
1   1 days 12:00:00  7.132183e+14  2.670615e+07  1.576982e+07  0.140300   
57  1 days 12:00:00  8.557534e+14  2.925326e+07  1.852192e+07  0.178335   
29  1 days 12:00:00  1.044811e+15  3.232354e+07  2.185825e+07  0.207326   
99  1 days 12:00:00  1.049182e+15  3.239107e+07  2.197634e+07  0.190805   
71  1 days 12:00:00  1.048381e+15  3.237871e+07  2.194473e+07  0.179720   
43  1 days 12:00:00  1.

INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 20.0.
INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 23.0.
INFO:fbprophet.diagnostics:Making 16 forecasts with cutoffs between 2018-01-23 12:00:00 and 2018-03-17 00:00:00
INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 17.0.


2             horizon           mse          rmse           mae      mape  \
35  1 days 00:00:00  1.388211e+15  3.725870e+07  3.076230e+07  0.217346   
63  1 days 00:00:00  1.279746e+15  3.577354e+07  2.839398e+07  0.204988   
77  1 days 00:00:00  1.271003e+15  3.565113e+07  2.816784e+07  0.202166   
105 1 days 00:00:00  1.107949e+15  3.328586e+07  2.577984e+07  0.182679   
21  1 days 00:00:00  1.147835e+15  3.387972e+07  2.716526e+07  0.199939   
91  1 days 00:00:00  6.521553e+14  2.553733e+07  2.308014e+07  0.208660   
15  1 days 12:00:00  8.490082e+14  2.913775e+07  2.493359e+07  0.200874   
1   1 days 12:00:00  1.078093e+15  3.283432e+07  2.897799e+07  0.223862   
57  1 days 12:00:00  1.070390e+15  3.271681e+07  2.871884e+07  0.222095   
29  1 days 12:00:00  1.037943e+15  3.221713e+07  2.774146e+07  0.213052   
99  1 days 12:00:00  1.012295e+15  3.181659e+07  2.703910e+07  0.202357   
71  1 days 12:00:00  8.990238e+14  2.998373e+07  2.396057e+07  0.180171   
43  1 days 12:00:00  1.

INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 20.0.
INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 23.0.
INFO:fbprophet.diagnostics:Making 16 forecasts with cutoffs between 2018-01-23 12:00:00 and 2018-03-17 00:00:00
INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 17.0.


...

8             horizon           mse          rmse           mae      mape  \
35  1 days 00:00:00  4.359628e+15  6.602748e+07  5.615735e+07  0.282340   
63  1 days 00:00:00  4.358036e+15  6.601543e+07  5.592590e+07  0.281733   
77  1 days 00:00:00  3.876197e+15  6.225911e+07  5.155358e+07  0.273894   
105 1 days 00:00:00  3.058737e+15  5.530585e+07  4.493167e+07  0.240994   
21  1 days 00:00:00  2.825795e+15  5.315820e+07  4.186216e+07  0.236496   
91  1 days 00:00:00  1.913473e+15  4.374326e+07  3.540404e+07  0.232200   
15  1 days 12:00:00  2.074431e+15  4.554592e+07  3.710349e+07  0.237620   
1   1 days 12:00:00  1.529591e+15  3.910998e+07  3.306423e+07  0.210830   
57  1 days 12:00:00  1.529790e+15  3.911253e+07  3.306986e+07  0.211684   
29  1 days 12:00:00  1.546507e+15  3.932565e+07  3.344499e+07  0.209064   
99  1 days 12:00:00  3.208445e+15  5.664314e+07  4.032920e+07  0.241417   
71  1 days 12:00:00  3.214589e+15  5.669735e+07  4.044292e+07  0.236702   
43  1 days 12:00:00  3.

INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 20.0.
INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 23.0.
INFO:fbprophet.diagnostics:Making 16 forecasts with cutoffs between 2018-01-23 12:00:00 and 2018-03-17 00:00:00
INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 17.0.


9             horizon           mse          rmse           mae      mape  \
35  1 days 00:00:00  3.536010e+14  1.880428e+07  1.606812e+07  0.369996   
63  1 days 00:00:00  3.449816e+14  1.857368e+07  1.539819e+07  0.345515   
77  1 days 00:00:00  3.288808e+14  1.813507e+07  1.472650e+07  0.316140   
105 1 days 00:00:00  5.218432e+14  2.284389e+07  1.792735e+07  0.330378   
21  1 days 00:00:00  5.314551e+14  2.305331e+07  1.821695e+07  0.328881   
91  1 days 00:00:00  5.196436e+14  2.279569e+07  1.746807e+07  0.314689   
15  1 days 12:00:00  5.456950e+14  2.336011e+07  1.811019e+07  0.283962   
1   1 days 12:00:00  1.196128e+15  3.458508e+07  2.494315e+07  0.323207   
57  1 days 12:00:00  1.238221e+15  3.518836e+07  2.641532e+07  0.379395   
29  1 days 12:00:00  1.058571e+15  3.253569e+07  2.258612e+07  0.335588   
99  1 days 12:00:00  1.053718e+15  3.246103e+07  2.246889e+07  0.342869   
71  1 days 12:00:00  1.081848e+15  3.289146e+07  2.329125e+07  0.351453   
43  1 days 12:00:00  1.

INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 20.0.
INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 23.0.


10             horizon           mse          rmse           mae      mape  \
35  1 days 00:00:00  1.815509e+15  4.260879e+07  3.575024e+07  0.259746   
63  1 days 00:00:00  1.853168e+15  4.304844e+07  3.731043e+07  0.273012   
77  1 days 00:00:00  1.774572e+15  4.212567e+07  3.548062e+07  0.269700   
105 1 days 00:00:00  1.754434e+15  4.188596e+07  3.470521e+07  0.267952   
21  1 days 00:00:00  1.669098e+15  4.085459e+07  3.330783e+07  0.279113   
91  1 days 00:00:00  1.308395e+15  3.617174e+07  2.941135e+07  0.271156   
15  1 days 12:00:00  9.339027e+14  3.055982e+07  2.525423e+07  0.226720   
1   1 days 12:00:00  1.206430e+15  3.473370e+07  2.888035e+07  0.246191   
57  1 days 12:00:00  1.469618e+15  3.833559e+07  3.088025e+07  0.275268   
29  1 days 12:00:00  1.544598e+15  3.930137e+07  3.144001e+07  0.284712   
99  1 days 12:00:00  1.525641e+15  3.905945e+07  3.099347e+07  0.265366   
71  1 days 12:00:00  1.598268e+15  3.997834e+07  3.299632e+07  0.269492   
43  1 days 12:00:00  1

In [437]:
for i in range(1, len(pm) + 1):
    print(i, pm[i][['mse']].mean(), pm[i][['mape']].mean())

1 mse    7.794492e+14
dtype: float64 mape    0.184619
dtype: float64
2 mse    9.802238e+14
dtype: float64 mape    0.186738
dtype: float64
3 mse    4.466868e+14
dtype: float64 mape    0.217522
dtype: float64
4 mse    3.015438e+15
dtype: float64 mape    0.234772
dtype: float64
5 mse    5.630637e+15
dtype: float64 mape    0.23573
dtype: float64
6 mse    4.079609e+14
dtype: float64 mape    0.2262
dtype: float64
7 mse    8.503684e+15
dtype: float64 mape    0.253915
dtype: float64
8 mse    4.105107e+15
dtype: float64 mape    0.247882
dtype: float64
9 mse    1.069176e+15
dtype: float64 mape    0.410304
dtype: float64
10 mse    2.152590e+15
dtype: float64 mape    0.274852
dtype: float64


In [439]:
temp_forecast_data = forecast_data.copy()

In [441]:
for i in range(1, len(temp_forecast_data) + 1):
    temp_forecast_data[i]['no. ATM'] = "K" + str(i)

In [442]:
temp_forecast_data[1]

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,extra_regressors_multiplicative,...,holidays_lower,holidays_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,take_money,take_money_lower,take_money_upper,yhat,no. ATM
0,2018-01-01,1.079938e+08,8.937236e+07,1.385472e+08,1.079938e+08,1.079938e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.054745,...,0.000000e+00,0.000000e+00,0.054745,0.054745,0.054745,0.054745,0.054745,0.054745,1.139059e+08,K1
1,2018-01-02,1.081058e+08,8.992581e+07,1.380278e+08,1.081058e+08,1.081058e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.054745,...,0.000000e+00,0.000000e+00,0.054745,0.054745,0.054745,0.054745,0.054745,0.054745,1.140240e+08,K1
2,2018-01-03,1.082178e+08,8.892716e+07,1.397467e+08,1.082178e+08,1.082178e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.054745,...,0.000000e+00,0.000000e+00,0.054745,0.054745,0.054745,0.054745,0.054745,0.054745,1.141422e+08,K1
3,2018-01-04,1.083298e+08,8.882350e+07,1.397469e+08,1.083298e+08,1.083298e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.054745,...,0.000000e+00,0.000000e+00,0.054745,0.054745,0.054745,0.054745,0.054745,0.054745,1.142603e+08,K1
4,2018-01-05,1.084419e+08,9.062689e+07,1.388826e+08,1.084419e+08,1.084419e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.054745,...,0.000000e+00,0.000000e+00,0.054745,0.054745,0.054745,0.054745,0.054745,0.054745,1.143785e+08,K1
5,2018-01-06,1.085539e+08,9.064462e+07,1.365274e+08,1.085539e+08,1.085539e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.054745,...,0.000000e+00,0.000000e+00,0.054745,0.054745,0.054745,0.054745,0.054745,0.054745,1.144966e+08,K1
6,2018-01-07,1.086659e+08,4.622307e+07,9.343467e+07,1.086659e+08,1.086659e+08,0.000000e+00,0.000000e+00,0.000000e+00,-0.358328,...,0.000000e+00,0.000000e+00,-0.358328,-0.358328,-0.358328,-0.358328,-0.358328,-0.358328,6.972787e+07,K1
7,2018-01-08,1.087779e+08,8.947322e+07,1.375812e+08,1.087779e+08,1.087779e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.054745,...,0.000000e+00,0.000000e+00,0.054745,0.054745,0.054745,0.054745,0.054745,0.054745,1.147329e+08,K1
8,2018-01-09,1.088899e+08,9.118909e+07,1.409089e+08,1.088899e+08,1.088899e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.054745,...,0.000000e+00,0.000000e+00,0.054745,0.054745,0.054745,0.054745,0.054745,0.054745,1.148511e+08,K1
9,2018-01-10,1.090020e+08,8.938748e+07,1.402802e+08,1.090020e+08,1.090020e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.054745,...,0.000000e+00,0.000000e+00,0.054745,0.054745,0.054745,0.054745,0.054745,0.054745,1.149692e+08,K1


In [443]:
for i in range(1, len(temp_forecast_data) + 1):
    temp_forecast_data[i] = temp_forecast_data[i].rename(columns={'ds': 'date'})

In [446]:
temp_forecast_data[1].tail(7)

,date,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,extra_regressors_multiplicative,...,holidays_lower,holidays_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,take_money,take_money_lower,take_money_upper,yhat,no. ATM
83,2018-03-25,1.058123e+08,4.543253e+07,9.271754e+07,1.058114e+08,1.058127e+08,0.000000e+00,0.000000e+00,0.000000e+00,-0.358328,...,0.000000e+00,0.000000e+00,-0.358328,-0.358328,-0.358328,-0.358328,-0.358328,-0.358328,6.789680e+07,K1
84,2018-03-26,1.057310e+08,8.829769e+07,1.350689e+08,1.057229e+08,1.057366e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.054745,...,0.000000e+00,0.000000e+00,0.054745,0.054745,0.054745,0.054745,0.054745,0.054745,1.115192e+08,K1
85,2018-03-27,1.056497e+08,8.475612e+07,1.357335e+08,1.056331e+08,1.056645e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.054745,...,0.000000e+00,0.000000e+00,0.054745,0.054745,0.054745,0.054745,0.054745,0.054745,1.114335e+08,K1
86,2018-03-28,1.055684e+08,9.009879e+07,1.415268e+08,1.055403e+08,1.055949e+08,3.823685e+06,3.823685e+06,3.823685e+06,0.054745,...,3.823685e+06,3.823685e+06,0.054745,0.054745,0.054745,0.054745,0.054745,0.054745,1.151714e+08,K1
87,2018-03-29,1.054871e+08,1.367767e+08,1.862278e+08,1.054479e+08,1.055255e+08,5.123359e+07,5.123359e+07,5.123359e+07,0.054745,...,5.123359e+07,5.123359e+07,0.054745,0.054745,0.054745,0.054745,0.054745,0.054745,1.624956e+08,K1
88,2018-03-30,1.054058e+08,2.187785e+08,2.678307e+08,1.053529e+08,1.054604e+08,1.326073e+08,1.326073e+08,1.326073e+08,0.054745,...,1.326073e+08,1.326073e+08,0.054745,0.054745,0.054745,0.054745,0.054745,0.054745,2.437835e+08,K1
89,2018-03-31,1.053245e+08,1.439159e+08,1.922593e+08,1.052519e+08,1.053965e+08,5.606990e+07,5.606990e+07,5.606990e+07,0.054745,...,5.606990e+07,5.606990e+07,0.054745,0.054745,0.054745,0.054745,0.054745,0.054745,1.671604e+08,K1


In [451]:
answer = {}

for i in range(1, len(temp_forecast_data) + 1):
    answer[i] = temp_forecast_data[i].loc[temp_forecast_data[i]['date'] > '2018-03-24']
    answer[i] = answer[i][['no. ATM', 'date', 'yhat']]
    answer[i] = answer[i].rename(columns={'yhat': 'prediction'})
    if i % 10 == 0:
        print(i)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
222

In [456]:
final_answer = pd.DataFrame()
final_answer_list = []

for i in range(1, len(answer) + 1):
    final_answer_list.append(answer[i])
    if i % 10 == 0:
        print(i)
        
final_answer = pd.concat(final_answer_list)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
222

In [461]:
final_answer.to_csv('result/prediction.csv', index=False)

In [464]:
pm[1]

,horizon,mse,rmse,mae,mape,coverage
35,1 days 00:00:00,1.204326e+15,3.470339e+07,2.652938e+07,0.218945,0.636364
63,1 days 00:00:00,1.126552e+15,3.356415e+07,2.414921e+07,0.194172,0.727273
77,1 days 00:00:00,1.105918e+15,3.325535e+07,2.311800e+07,0.185707,0.727273
105,1 days 00:00:00,9.192885e+14,3.031977e+07,1.951526e+07,0.154925,0.818182
21,1 days 00:00:00,7.992215e+14,2.827051e+07,1.659158e+07,0.146958,0.909091
91,1 days 00:00:00,1.020820e+14,1.010356e+07,8.692134e+06,0.113619,1.000000
15,1 days 12:00:00,1.378558e+14,1.174120e+07,9.805787e+06,0.117029,1.000000
1,1 days 12:00:00,7.132183e+14,2.670615e+07,1.576982e+07,0.140300,0.909091
57,1 days 12:00:00,8.557534e+14,2.925326e+07,1.852192e+07,0.178335,0.818182
29,1 days 12:00:00,1.044811e+15,3.232354e+07,2.185825e+07,0.207326,0.727273
